---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [47]:
import numpy as np
import pandas as pd
import re
from scipy.stats import ttest_ind
from datetime import datetime, timedelta

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
states1={}
for i,j in states.items():
    states1[j]=i 

In [146]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    file=open("university_towns.txt")
    ls=[]           #empty list for the dataframe
    statename=None             # statename
    for line in file:
        line=line.rstrip()
        if len(re.findall('(.+)\[edit\]',line))==1: #this line must be the statename
            statename=re.findall('(.+)\[edit\]',line)[0]
        elif re.search('[(]',line)!= None:
            ls.append([statename,line.split("(")[0].rstrip()])
        else:
            ls.append([statename,line])
    #there three are super irregular,so input by hand
    df=pd.DataFrame(ls,columns=["State", "RegionName"])    
     ### replace state     df.replace({"State":states1},inplace=True)
    return df
get_list_of_university_towns().head()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


In [44]:
    GDP=pd.read_excel("gdplev.xls",skiprows=4,usecols=[4,5,6]) ##read GDP file and clean data
    GDP.columns=GDP.iloc[0]
    GDP=GDP.iloc[3:]
    GDP.columns=[                            'Quarters',
            'GDP in billions of current dollars',
       'GDP in billions of chained 2009 dollars']
    GDP=GDP[GDP['Quarters']>='2000q1']
    GDP.set_index(GDP['Quarters'],inplace=True)
    GDP.head()

,Quarters,GDP in billions of current dollars,GDP in billions of chained 2009 dollars
Quarters,,,
2000q1,2000q1,10031,12359.1
2000q2,2000q2,10278.3,12592.5
2000q3,2000q3,10357.4,12607.7
2000q4,2000q4,10472.3,12679.3
2001q1,2001q1,10508.1,12643.3


In [43]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''

    s=GDP['GDP in billions of chained 2009 dollars']
    for i in range(len(s))[1:-2]:
        if s[i]<s[i-1] and s[i+1]<s[i]:
            return s.index[i]
get_recession_start()

'2008q3'

In [56]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    s=GDP['GDP in billions of chained 2009 dollars'] 
    for i in range(len(s)):
        if s.index[i]<= '2008q3': continue
        if s[i]>s[i-1] and s[i-1]>s[i-2]:
            return s.index[i]
get_recession_end()

'2009q4'

In [58]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    s=GDP['GDP in billions of chained 2009 dollars']
    re_bottom='2008q3'
    for i in range(len(s)):
        if s.index[i]<='2008q3' or s.index[i]>='2009q4': continue
        if s[i]<s[re_bottom]:
            re_bottom=s.index[i]
    return re_bottom
get_recession_bottom()

'2009q2'

In [68]:
pd.PeriodIndex([datetime.strptime("1996-07","%Y-%m")],freq="Q")

PeriodIndex(['1996Q3'], dtype='period[Q-DEC]', freq='Q-DEC')

In [157]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df=pd.read_csv("City_Zhvi_AllHomes.csv")
    df.replace({'State':states},inplace=True)
    df=df.set_index([df['State'],df['RegionName']])
    df=df[df.columns[6:]]
    df.columns=pd.PeriodIndex([datetime.strptime(date,"%Y-%m") for date in df.columns],freq="Q")
    df=df.T
    df=df.iloc[df.index>="2000q1"]
    df=df.groupby(df.index).mean().T
    return df
len(convert_housing_data_to_quarters())

10730

In [163]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    df=convert_housing_data_to_quarters()
    p_s=df['2008q2']
    p_b=df['2009q2']
    utown=get_list_of_university_towns()
    utown.set_index([utown["State"],utown['RegionName']],inplace=True)
    ratio=(p_s/p_b).to_frame()
    ratio.columns=['ratio']
    utown_r=pd.merge(utown,ratio,how="left",left_index=True,right_index=True)['ratio'].dropna()
    nutown_r=ratio.drop(utown_r.index)['ratio'].dropna()
    if utown_r.mean()<nutown_r.mean(): better="university town"
    else: better='non-university town'
    t=ttest_ind(utown_r,nutown_r)
    if t[1]<0.01: different=True
    else: diferent=False
    return (different,t[1],better) 
run_ttest()

(True, 0.002724063704761164, 'university town')